In [23]:
import cv2 as cv
import numpy as np
import os
from PIL import ImageFont, ImageDraw, Image
from datetime import datetime

In [24]:
def select_camera():
    """
    사용자가 관측 지점을 선택하고 해당 카메라의 VideoCapture 객체를 반환하는 함수
    Returns:
        tuple: 선택된 관측 지점 이름과 해당 카메라의 VideoCapture 객체
    """
    # 카메라 정보 리스트
    cameras = [
        {"name": "불당현대 사거리", "url": "rtsp://210.99.70.120:1935/live/cctv048.stream"},
        {"name": "시청 사거리", "url": "rtsp://210.99.70.120:1935/live/cctv045.stream"},
        {"name": "예술의 전당", "url": "rtsp://210.99.70.120:1935/live/cctv036.stream"}
    ]

    # 사용자에게 보여줄 카메라 선택 목록 생성 및 출력
    prompt = "관측 지점 선택: " + ", ".join([f"[{i+1}: {cam['name']}]" for i, cam in enumerate(cameras)])
    print(prompt)

    point = 0
    try:
        point = int(input("관측 지점 번호 입력: "))
    except ValueError:
        pass # 유효하지 않은 입력은 아래에서 기본값으로 처리

    # 사용자가 입력한 번호가 유효한 범위 내에 있는지 검증
    if not (1 <= point <= len(cameras)):
        print(f"잘못된 입력입니다. [1: {cameras[0]['name']}] 지점을 관측 지점으로 설정합니다.")
        point = 1

    selected_camera = cameras[point - 1]
    point_name = selected_camera["name"]
    rtsp_url = selected_camera["url"]

    print(f"{point}번 지점({point_name}) 선택됨.")
    # 선택된 카메라 이름과 VideoCapture 객체를 튜플로 반환
    return point_name, cv.VideoCapture(rtsp_url)

In [25]:
def apply_filter(frame, filter_type):
    """
    필터 타입에 따라 프레임에 필터를 적용하는 함수
    filter_type:
        1 - 필터 없음
        2 - 그레이 스케일 필터
        3 - 가우시안 블러 필터
        4 - 엣지 검출 필터
        5 - 세피아 필터
        6 - 좌우 반전 (거울 효과)
    """
    if filter_type == 1:
        return frame
    elif filter_type == 2:
        return cv.cvtColor(frame, cv.COLOR_BGR2GRAY)
    elif filter_type == 3:
        return cv.GaussianBlur(frame, (7, 7), 0)
    elif filter_type == 4:
        return cv.Canny(frame, 100, 150)
    elif filter_type == 5:
        kernel = np.array([[0.273, 0.535, 0.132],
                            [0.349, 0.686, 0.165],
                            [0.391, 0.769, 0.180]])
        return cv.transform(frame, kernel)
    elif filter_type == 6:
        return cv.flip(frame, 1)
    return frame

In [26]:
point_name, cap = select_camera()

# 영상 처리 관련 설정
TARGET_WIDTH = 720
TARGET_HEIGHT = 480
is_recording = False
fourcc = cv.VideoWriter_fourcc(*'XVID')
fps = 20.0
current_filter = 1  # 기본 필터: 1 (필터 없음)

# 녹화 영상 저장 경로 및 파일명 설정
save_path = './videos/'  # 저장 경로
if not os.path.exists(save_path):
    os.makedirs(save_path)  # 폴더가 없으면 생성

output_filename = os.path.join(save_path, f"{point_name.replace(' ', '_')}_output_{datetime.now().strftime("%Y%m%d%H%M%S")}.avi")

# 카메라 연결 상태 확인
if not cap.isOpened():
    print("카메라 연결 실패")
    exit()

관측 지점 선택: [1: 불당현대 사거리], [2: 시청 사거리], [3: 예술의 전당]
2번 지점(시청 사거리) 선택됨.


In [28]:
"""
키보드 입력 안내
- ESC: 프로그램 종료
- Space: 녹화 시작/중지
- 숫자 1~6: 필터 변경
    1: 필터 없음
    2: 그레이 스케일
    3: 가우시안 블러
    4: 엣지 검출
    5: 세피아
    6: 좌우 반전
"""

while True:
    ret, frame = cap.read()
    if not ret:
        print("프레임 읽기 실패")
        break
    
    frame = cv.resize(frame, (TARGET_WIDTH, TARGET_HEIGHT))
    h, w = frame.shape[:2]
    
    # 현재 설정된 필터를 프레임에 적용
    frame_filtered = apply_filter(frame, current_filter)

    # 화면에 표시할 텍스트(관측 지점, 시간) 설정 - 한글 지원
    fontpath = "c:/Windows/Fonts/NanumBarunGothic.ttf"
    font = ImageFont.truetype(fontpath, 20)
    frame_pil = Image.fromarray(cv.cvtColor(frame_filtered, cv.COLOR_BGR2RGB))
    draw = ImageDraw.Draw(frame_pil)
    draw.text((w-700, h-30), point_name, font=font, fill=(0, 255, 0))
    draw.text((w-700, h-60), datetime.now().strftime("%Y-%m-%d %H:%M:%S"), font=font, fill=(0, 255, 0))
    frame_filtered = cv.cvtColor(np.array(frame_pil), cv.COLOR_RGB2BGR)
    frame_ui = frame_filtered.copy()
    
    # 녹화 상태에 따른 처리
    if is_recording: # 녹화 중일 때
        if 'out' not in locals(): # VideoWriter 객체가 없으면 생성
            out = cv.VideoWriter(output_filename, fourcc, fps, (w, h))
        out.write(frame_filtered)
        cv.putText(frame_ui, "REC", (10, 30), cv.FONT_HERSHEY_SIMPLEX, 1, (0, 0, 255), 2) # "REC" 표시 - 영문
    else: # 녹화 중이 아닐 때
        if 'out' in locals(): # VideoWriter 객체가 존재하면
            out.release() # 리소스 해제
            del out # 객체 삭제
            print(f"녹화된 파일이 저장되었습니다: {output_filename}")

    cv.imshow('Traffic Lens', frame_ui)

    # 키보드 입력 대기
    key = cv.waitKey(1) & 0xFF
    if key == 27: # ESC 키 입력 시
        cv.destroyAllWindows()
        break

    elif key == 32:  # Space 키 입력 시
        if not is_recording:
            is_recording = True
            print("녹화 시작")
        else:
            is_recording = False
            print("녹화 중지")

    elif key in [ord(str(i)) for i in range(1, 7)]:  # 숫자 키 1~6 입력 시
        current_filter = int(chr(key))
        print(f"필터 {current_filter}로 변경됨.")

필터 4로 변경됨.
녹화 시작
녹화 중지
녹화된 파일이 저장되었습니다: ./videos/시청_사거리_output_20250916200135.avi
